We use Z-Score Standardization to scale our data 

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset
df = pd.read_csv("final_tech_adoption_index_data.csv", index_col=0)

# Step 2: Initialize the StandardScaler
scaler = StandardScaler()

# Step 3: Fit and transform the data
zscore_scaled = scaler.fit_transform(df)

# Step 4: Convert the result back to a DataFrame
df_zscore = pd.DataFrame(zscore_scaled, columns=df.columns, index=df.index)

# Step 5: Preview the result
print(df_zscore.to_string())

# Optional: Save the standardized data
df_zscore.to_csv("tech_adoption_zscore.csv")


               Internet_Users_Pct  Fixed_Broadband_per_100  R&D_Expenditure_Pct_GDP  Mobile_Cellular_per_100  High_Technology_Exports_USD  ICT_Graduates_Pct  Problem_Solving_Tech_Level3_Pct  B2C_Index_2020  DAI_Overall  DAI_Business  DAI_People   DAI_Gov
Country Name                                                                                                                                                                                                                                                  
United States           -0.140575                 0.210816                 1.069450                -0.863663                     2.244790          -0.516384                        -0.177795        0.431718    -0.208791     -0.802425   -0.267266  0.225745
Denmark                  1.323302                 1.262288                 0.490707                 0.214851                    -0.412011          -0.093020                         0.366061        0.942558     0.425243      1.333502   

Check for outliers

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Step 1: Load the dataset
df = pd.read_csv("final_tech_adoption_index_data.csv", index_col=0)

# Step 2: Standardize using Z-score
scaler = StandardScaler()
df_zscore = pd.DataFrame(scaler.fit_transform(df), columns=df.columns, index=df.index)

# Step 3: Identify outliers (|Z| > 3)
outlier_mask = df_zscore.abs() > 3
outlier_values = df_zscore[outlier_mask]

# Step 4: Flatten the DataFrame to show only outlier values
outlier_data_only = outlier_values.stack().reset_index()
outlier_data_only.columns = ["Country", "Feature", "Z-Score"]

# Step 5: Display or export
print(outlier_data_only)

# Optional: Save to CSV
# outlier_data_only.to_csv("zscore_outliers.csv", index=False)


   Country                      Feature   Z-Score
0   Israel      R&D_Expenditure_Pct_GDP  3.093380
1  Germany  High_Technology_Exports_USD  3.029286


We can see Israe's (RnD Percentage per GDP) is an outlier but after fact checking, it is correct so we are keeping it.

That is also the case for Germany as the top High-Tech_Export outlier among these countries so we are keeping it as well.